<a href="https://colab.research.google.com/github/JosenildoJunior/StatPyDataScience/blob/main/Regress%C3%A3o_Log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regressão logistica**

## **Conceitos importantes**

Antes de partirmos para a regressão propriamente dito, vamos relembrar alguns pontos sobre a regressão linear

**Regressão Linear**:

- Definição:

  - **A regressão linear** é um método estatístico usado para modelar a relação entre uma variável dependente (ou resposta) e uma ou mais variáveis independentes (ou características).

- Objetivo:

  - O objetivo da regressão linear é prever valores contínuos com base em uma relação linear entre as características e a variável de destino.

- Exemplo:

  - Um exemplo comum de regressão linear é prever o preço de uma casa com base em características como área, número de quartos, localização, etc.

- Saída:

  - A saída de um modelo de regressão linear é um valor contínuo. Por exemplo, o preço previsto de uma casa em dólares.

Agora vamos observar alguns pontos sobre a classificação

**Classificação**:
- Definição:

  - **A classificação** é um tipo de algoritmo de aprendizado de máquina supervisionado usado para prever a classe de um objeto com base em suas características.
- Objetivo:

  - O objetivo da classificação é separar os pontos de dados em diferentes categorias ou classes usando uma linha reta (ou plano) no espaço de características.

- Exemplo:

  - Um exemplo comum de classificação linear é a classificação de flores em diferentes espécies com base em características como comprimento e largura das pétalas e sépalas.

- Saída:

  - A saída de um modelo de classificação é uma classe ou categoria discreta. Por exemplo, "espécie 1", "espécie 2", etc.



Dessa forma podemos observar que são tecnicas bem distintas uma das outras partidno desde o seu objetivo até a sua saida

## **Prática**

Agora que já revimos e entendemos um pouco mais sobre classificação vamos aplicar na pratica, para isso como de costume vamos importar as nossas biliotecas e nossos dados

### **Carregando os dados**

In [ ]:
# Acesso ao drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Manipulação de dados
import pandas as pd

# Algébra linear
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Funções estatísticas
import statistics
import scipy.stats

# Machine learning
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as skl_lm
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import train_test_split

Agora que já importamos todas as bibliotecas necessarias vamos importar a base de dados propriamente dita

In [ ]:
# Importando o dataset
df = pd.read_csv('/content/drive/MyDrive/Estatística para ciência de dados/data_classificacao.csv')

Agora que já carregamos a base de dados vamos ter um primeiro contato com esses dados



In [ ]:
# Observando os primeiros registros
df.head()

Vemos diversas variaveis aqui, que até o momento não sabemos do que se trata então vamos observar o dicionario de dados


* **id**
* **diagnosis**: M = maligno, B = benigno

*Colunas 3 a 32*

Dez características de valor real são computadas para cada núcleo celular:

* **radius**: distâncias do centro aos pontos do perímetro
* **texture**: desvio padrão dos valores da escala de cinza
* **perimeter**
* **area**
* **smoothness**: variação local nos comprimentos dos raios
* **compactness**: perímetro^2 / área - 1,0
* **concavity**: severidade das porções côncavas do contorno
* **concave points**: número de porções côncavas do contorno
* **symmetry**
* **fractal dimension**: "aproximação da costa" - 1